In [1]:
from pandas_datareader import data as pdr
import pandas as pd
import numpy as np
import cvxpy as cvx
from scipy.stats import norm
from numpy.linalg import inv
import math

import keras
import seaborn as sns
import sklearn
import tensorflow as tf

# for plotting
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
import matplotlib.pyplot as plt

# Initialize plotly offline
plotly.offline.init_notebook_mode(connected=True)

from benchmarks import *
from datafunctions import *

Using TensorFlow backend.


### Homework: calculate the development of CRP for bandit & risk-aware portfolio
- Change the return to log(return) when optimizing
- Play with the weight $\lambda$: $w_t^* = \lambda w_t^M+(1-\lambda)w_t^C$

In [2]:
def Balanced_Portfolio(weight,stock_returns,wM,wC):
    asset_returns = stock_returns.as_matrix()
    T, m = asset_returns.shape
    T = T + 1
    
    w = weight*wM + (1-weight)*wC
    wealth_factors = np.ones(T)
    for i in range(1,T):
        wealth_factors[i] = wealth_factors[i-1]*np.dot(w, asset_returns[i-1,:])
        
    return w, wealth_factors

### Visualization demo
After having the output $\mathtt{wealth\_factors}$ (cumulative reward) from the algorithms above, we do the visualization as follows.

In [3]:
us_stock_prices = pd.read_csv('./00_Data/us_stocks_latest.csv',
                              header=0,
                              index_col=0)
us_stock_prices.index = pd.to_datetime(us_stock_prices.index)

#Process Stock Data
stock_prices_norm, stock_returns = process_stock_data(us_stock_prices)

# initialize with SPY
SPY_benchmark = stock_prices_norm.drop(stock_prices_norm.columns[1:], axis=1).iloc[1:]

# # Solves the convex optimization problem of the Best constant rebalanced portfolio in hindsight
if all(stock_returns.iloc[0].isna()):
        stock_returns = stock_returns.iloc[1:]
BCRP, SPY_benchmark['BCRP_reward'] = bcrp_wealth_factors(stock_returns)

# # Uniform Constant Rebalanced Portfolio
SPY_benchmark['UCRP_reward'] = ucrp_wealth_factors(stock_returns)

# # Add reward for other portfolios as well, for example:

# w, SPY_benchmark['Balanced_Portfolio'] = Balanced_Portfolio(weight, stock_returns, wM, wC)

# # plot
iplot(SPY_benchmark.iplot(asFigure=True, kind='scatter',xTitle='Dates',yTitle='Wealth Factor',title='Relative Wealth over time'))